In [1]:
from smartsim import Experiment
from smartsim.database import SlurmOrchestrator
from smartsim.settings import SrunSettings
from smartsim import slurm

def launch_cluster_orc(experiment, port, alloc):
    """Just spin up a database cluster, check the status
       and tear it down"""

    db = SlurmOrchestrator(port=port,
                            db_nodes=1,
                            batch=False,
                            alloc=alloc,
                            interface="ib0")
    

    # generate directories for output files
    # pass in objects to make dirs for
    experiment.generate(db, overwrite=True)

    # start the database on interactive allocation
    experiment.start(db, block=True)

    return db

def create_uploader(experiment, alloc, nodes=1, tasks_per_node=1):

    srun = SrunSettings(exe="python",
                        exe_args="data_uploader.py",
                        env_vars={"PYTHONUNBUFFERED": "1"},
                        alloc=alloc)
    srun.set_nodes(nodes)
    srun.set_tasks_per_node(tasks_per_node)

    uploader = experiment.create_ensemble("uploader", replicas=2, run_settings=srun)

    # create directories for the output files and copy
    # scripts to execution location inside newly created dir
    # only necessary if its not an executable (python is executable here)
    uploader.attach_generator_files(to_copy=["./tf/data_uploader.py"])
    experiment.generate(uploader, overwrite=True)
    return uploader


def create_trainer(experiment, alloc):

    srun = SrunSettings(exe="python",
                        exe_args="training_service.py",
                        env_vars={"PYTHONUNBUFFERED": "1"},
                        alloc=alloc)
    srun.set_tasks(1)

    trainer = experiment.create_model("trainer", srun)

    # create directories for the output files and copy
    # scripts to execution location inside newly created dir
    # only necessary if its not an executable (python is executable here)
    trainer.attach_generator_files(to_copy="./tf/training_service.py")
    experiment.generate(trainer, overwrite=True)
    return trainer


def create_trainer_hvd(experiment, alloc, nodes=1, tasks_per_node=1):

    srun = SrunSettings(exe="python",
                        exe_args="training_service_hvd.py",
                        env_vars={"PYTHONUNBUFFERED": "1"},
                        alloc=alloc)
    srun.set_nodes(nodes)
    srun.set_tasks_per_node(tasks_per_node)

    trainer = experiment.create_model("trainer", srun)

    # create directories for the output files and copy
    # scripts to execution location inside newly created dir
    # only necessary if its not an executable (python is executable here)
    trainer.attach_generator_files(to_copy="./tf/training_service_hvd.py")
    experiment.generate(trainer, overwrite=True)
    return trainer

In [2]:
alloc = slurm.get_allocation(nodes=4, time="03:00:00", options={"constraint": "V100", "partition": "spider"})

12:35:45 osprey.us.cray.com SmartSim[32669] INFO Allocation successful with Job ID: 529607


In [3]:
exp = Experiment("launch_streaming", launcher="slurm")

db_port = 6780

# start the database
db = launch_cluster_orc(exp, db_port, alloc)
uploader_model = create_uploader(exp, alloc, 1, 2)
uploader_model.enable_key_prefixing()
exp.start(uploader_model, block=False, summary=False)
trainer_model = create_trainer(exp, alloc)
for uploader in uploader_model.entities:
    trainer_model.register_incoming_entity(uploader)

exp.start(trainer_model, block=True, summary=False)

# shutdown the database because we don't need it anymore
exp.stop(db)

print(exp.summary())

11:11:08 osprey.us.cray.com SmartSim[139348] INFO Working in previously created experiment
11:11:41 osprey.us.cray.com SmartSim[139348] INFO Working in previously created experiment
11:11:48 osprey.us.cray.com SmartSim[139348] INFO Working in previously created experiment
11:11:56 osprey.us.cray.com SmartSim[139348] INFO uploader_0(529604.1): New
11:11:56 osprey.us.cray.com SmartSim[139348] INFO uploader_1(529604.2): New
11:11:56 osprey.us.cray.com SmartSim[139348] INFO trainer(529604.3): New
11:12:01 osprey.us.cray.com SmartSim[139348] INFO uploader_0(529604.1): Running
11:12:01 osprey.us.cray.com SmartSim[139348] INFO uploader_1(529604.2): Running
11:12:01 osprey.us.cray.com SmartSim[139348] INFO trainer(529604.3): Running
11:12:06 osprey.us.cray.com SmartSim[139348] INFO uploader_0(529604.1): Running
11:12:06 osprey.us.cray.com SmartSim[139348] INFO uploader_1(529604.2): Running
11:12:06 osprey.us.cray.com SmartSim[139348] INFO trainer(529604.3): Running
11:12:11 osprey.us.cray.com 

KeyboardInterrupt: 

In [6]:
exp.stop(db, uploader_model, trainer_model)

KeyError: 

In [3]:
exp = Experiment("launch_streaming_hvd", launcher="slurm")

db_port = 6780

# start the database
db = launch_cluster_orc(exp, db_port, alloc)
uploader_model = create_uploader(exp, alloc, 1, 8)
uploader_model.enable_key_prefixing()
exp.start(uploader_model, block=False, summary=False)
trainer_model = create_trainer_hvd(exp, alloc, 1, 8)
for uploader in uploader_model.entities:
    trainer_model.register_incoming_entity(uploader)

exp.start(trainer_model, block=True, summary=False)

# shutdown the database because we don't need it anymore
exp.stop(db)

print(exp.summary())

12:49:22 osprey.us.cray.com SmartSim[32669] INFO Working in previously created experiment
12:49:29 osprey.us.cray.com SmartSim[32669] INFO Working in previously created experiment
12:49:37 osprey.us.cray.com SmartSim[32669] INFO uploader_0(529607.1): New
12:49:37 osprey.us.cray.com SmartSim[32669] INFO uploader_1(529607.2): New
12:49:37 osprey.us.cray.com SmartSim[32669] INFO trainer(529607.3): New
12:49:42 osprey.us.cray.com SmartSim[32669] INFO uploader_0(529607.1): Running
12:49:42 osprey.us.cray.com SmartSim[32669] INFO uploader_1(529607.2): Running
12:49:42 osprey.us.cray.com SmartSim[32669] INFO trainer(529607.3): Running
12:49:47 osprey.us.cray.com SmartSim[32669] INFO uploader_0(529607.1): Running
12:49:47 osprey.us.cray.com SmartSim[32669] INFO uploader_1(529607.2): Running
12:49:47 osprey.us.cray.com SmartSim[32669] INFO trainer(529607.3): Running
12:49:52 osprey.us.cray.com SmartSim[32669] INFO uploader_0(529607.1): Running
12:49:52 osprey.us.cray.com SmartSim[32669] INFO up

KeyboardInterrupt: 

In [4]:
slurm.release_allocation(alloc)

12:58:48 osprey.us.cray.com SmartSim[32669] INFO Releasing allocation: 529607
12:58:48 osprey.us.cray.com SmartSim[32669] INFO Successfully freed allocation 529607
12:58:55 osprey.us.cray.com SmartSim[32669] INFO uploader_0(529607.1): Cancelled
12:58:55 osprey.us.cray.com SmartSim[32669] INFO uploader_1(529607.2): Cancelled
12:58:55 osprey.us.cray.com SmartSim[32669] INFO trainer(529607.3): Cancelled
12:58:55 osprey.us.cray.com SmartSim[32669] INFO orchestrator_0(529607.0): Cancelled
